# start

In [1]:
import numpy as np
import plotly.graph_objects as go
import torch
from itertools import product

# plotting tools

In [2]:
def make_links_for_mask(mask, marker, line, **scatter_kwargs):
    assert mask.shape[0] < 4
    assert len(mask.shape[1:]) == mask.shape[0] # Nd == Nd
    if isinstance(mask, torch.Tensor):
        mask = mask.detach().to_numpy()
    mask = np.asarray(mask)
    lines = []
    for mask_idx in product(*[range(L) for L in mask.shape]):
        if mask[mask_idx] == 0:
            continue
        mu, *x0 = mask_idx
        x0 = list(x0)
        x1 = list(x0)
        x1[mu] += 1
        if mask.shape[0] == 2:
            x0 += [0]
            x1 += [0]
        lines.append(go.Scatter3d(
            x=[x0[0],x1[0]], y=[x0[1],x1[1]], z=[x0[2],x1[2]],
            line=line, marker=marker,
            **scatter_kwargs))
    return lines

def make_plaq(x, mu, nu, shrink=0.975, **mesh_kwargs):
    x = np.array(x)
    muhat = np.zeros_like(x); muhat[mu] = 1
    nuhat = np.zeros_like(x); nuhat[nu] = 1
    corners = np.array([x, x+muhat, x+muhat+nuhat, x+nuhat]).T
    center = np.mean(corners, axis=1).reshape(-1,1)
    corners = (corners-center)*shrink + center
    tri_vtx_idxs = np.array([
        [0,1,2],
        [2,3,0]
    ]).T
    if len(x) == 3:
        plaq = go.Mesh3d(
            x=corners[0], y=corners[1], z=corners[2],
            i=tri_vtx_idxs[0], j=tri_vtx_idxs[1], k=tri_vtx_idxs[2],
            **mesh_kwargs)
    else:
        plaq = go.Mesh3d(
            x=corners[0], y=corners[1],
            i=tri_vtx_idxs[0], j=tri_vtx_idxs[1], k=tri_vtx_idxs[2],
            **mesh_kwargs)
    return plaq

def make_plaqs_for_plane_mask(
        mask, mu, nu, offset, shrink=0.975, **mesh_kwargs):
    if isinstance(mask, torch.Tensor):
        mask = mask.detach().numpy()
    mask = np.asarray(mask)
    plaqs = []
    offset_dir = list({0,1,2} - {mu,nu})[0]
    for coords in product(*[range(L) for L in mask.shape]):
        x = np.zeros(3)
        x[mu] = coords[0]; x[nu] = coords[1]
        x[offset_dir] = offset
        if mask[coords] == 0:
            continue
        plaqs.append(make_plaq(x,mu,nu, shrink=shrink, **mesh_kwargs))
    return plaqs

def make_plaqs_for_mask(
        mask, plaq_channel_directions=((0,1),(1,2),(0,2)), **mesh_kwargs):
    if len(mask.shape) == 2:
        return make_plaqs_for_plane_mask(mask, 0, 1, 0, **mesh_kwargs)
    if isinstance(mask, torch.Tensor):
        mask = mask.detach().numpy()
    assert len(mask.shape) == 4
    plaqs = []
    for mn_idx, (mu,nu) in enumerate(plaq_channel_directions):
        offset_dir = [x for x in (0,1,2) if x not in (mu,nu)][0]
        all_planes = mask[mn_idx]
        all_planes = np.moveaxis(all_planes, (offset_dir,mu,nu), (0,1,2))
        for offset, plane_mask in enumerate(all_planes):
            plaqs += make_plaqs_for_plane_mask(
                plane_mask,mu,nu,offset,**mesh_kwargs)
    return plaqs

def plaq_channel_directions(d):
    return [(i,j) for i in range(d-1) for j in range(i+1,d)]

def plot_masks(link_masks, plaq_masks, plaq_dirs, opacity=0.5):
    Lx, Ly, Lz = link_masks[0].shape[1:]
    colors = ['green', 'red', 'blue', 'orange']
    data = []
    for i, (link_mask, plaq_mask, _plaq_dirs) \
      in enumerate(zip(link_masks, plaq_masks, plaq_dirs)):
        data += make_links_for_mask(
            link_mask,
            marker=None,
            line=dict(color='black', width=8),
            mode='lines') + \
          make_plaqs_for_mask(
            plaq_mask,
            plaq_channel_directions=_plaq_dirs,
            opacity=opacity,
            color=colors[i%4])
    fig = go.Figure(data=data)
    fig.update_layout(
        showlegend=False,
        scene_xaxis=dict(
            title='', tickwidth=0, tickvals=list(range(Lx)), ticktext=('',)*Lx),
        scene_yaxis=dict(
            title='', tickwidth=0, tickvals=list(range(Ly)), ticktext=('',)*Ly),
        scene_zaxis=dict(
            title='', tickwidth=0, tickvals=list(range(Lz)), ticktext=('',)*Lz),
        width=1000, height=800)
    fig.show()

# link mask

In [3]:
def make_link_mask(
        dimensions,
        orientations,
        width,
        active_phase,
        orients_to_shift,
        shifts):
    dim = len(dimensions)
    coords = np.stack(
        np.meshgrid(
            *list(map(np.arange, dimensions)),
            indexing='ij'),
        axis=-1)
    phases = np.sum(
        coords[..., orients_to_shift] * shifts, axis=-1) % width
    site_mask = (phases == active_phase)
    direction_mask = np.array([k in orientations for k in range(dim)])
    link_mask = site_mask[..., np.newaxis] * direction_mask
    return np.rollaxis(link_mask, -1).astype(int)

# loop mask

In [4]:
def infer_active_and_frozen(link_mask, loop_types, loop_fn):
    num_links = np.count_nonzero(link_mask)
    link_coords = (np.arange(num_links),) + np.where(link_mask)
    split_links = np.zeros((num_links,) + link_mask.shape)
    split_links[link_coords] = 1

    loops = np.abs(loop_fn(split_links, loop_types)).sum(axis=0)
    frozen_loop_mask = (loops == np.zeros_like(loops)).astype(int)
    sign = loop_fn(link_mask[np.newaxis,], loop_types).squeeze(0)

    num_loops, dim = len(loop_types), link_mask.shape[0]
    active_loop_mask, active_loop_ind = [], []
    for d in range(dim):
        for k in range(num_loops):
            if d not in loop_types[k]:
                continue
            if np.all(link_mask[d] == 0):
                active_loop_mask.append(link_mask[d])
                active_loop_ind.append(-1)
                break
            candidate = link_mask[d] * loops[k]
            if np.all(candidate == link_mask[d]):
                active_loop_mask.append(candidate * sign[k])
                active_loop_ind.append(k)
                break
    active_loop_mask = np.stack(active_loop_mask, axis=0)

    return active_loop_mask, active_loop_ind, frozen_loop_mask

def plaq_fn(links, loop_types):
    plaqs = []
    for mu,nu in loop_types:
        plaqs.append(
            links[:,mu]
            + np.roll(links[:,nu], -1, axis=mu+1)
            - np.roll(links[:,mu], -1, axis=nu+1)
            - links[:,nu])
    return np.stack(plaqs, axis=1)

# alternation

In [5]:
def alternate_params(
        active_phase,
        width,
        dimensions,
        orientations,
        orients_to_shift):
    active_phase += 1
    if active_phase == width:
        dim = len(dimensions)
        orientations = [(k + 1) % dim for k in orientations]
        orients_to_shift = [(k + 1) % dim for k in orients_to_shift]
    active_phase %= width
    return active_phase, orientations, orients_to_shift

# plot 1

In [6]:
link_mask = make_link_mask(
    dimensions = [6, 6, 4],
    orientations = [2],
    width = 4,
    active_phase = 0,
    orients_to_shift = [0, 1],
    shifts = [1, 2])
plaq_mask, plaq_index, _ = infer_active_and_frozen(
    link_mask,
    plaq_channel_directions(link_mask.shape[0]),
    plaq_fn)
plaq_dirs = np.array(plaq_channel_directions(link_mask.shape[0]))[plaq_index]
plot_masks([link_mask], [plaq_mask], [plaq_dirs])

# plot 2

In [7]:
link_mask = make_link_mask(
    dimensions = [6, 6, 4],
    orientations = [2],
    width = 4,
    active_phase = 0,
    orients_to_shift = [2, 0, 1],
    shifts = [2, 1, 2])
plaq_mask, plaq_index, _ = infer_active_and_frozen(
    link_mask,
    plaq_channel_directions(link_mask.shape[0]),
    plaq_fn)
plaq_dirs = np.array(plaq_channel_directions(link_mask.shape[0]))[plaq_index]
plot_masks([link_mask], [plaq_mask], [plaq_dirs])

# plot 3

In [8]:
dimensions = [4, 4, 4]
orientations = [2]
width = 4
active_phase = 0
orients_to_shift = [0, 1]
shifts = [1, 2]

link_masks, plaq_masks, plaq_dirs = [], [], []

for i in range(12):
    link_masks.append(make_link_mask(
        dimensions=dimensions,
        orientations=orientations,
        width=width,
        active_phase=active_phase,
        orients_to_shift=orients_to_shift,
        shifts=shifts))
    plaq_mask, plaq_index, _ = infer_active_and_frozen(
        link_masks[-1],
        plaq_channel_directions(link_masks[-1].shape[0]),
        plaq_fn)
    plaq_masks.append(plaq_mask)
    plaq_dirs.append(
        np.array(plaq_channel_directions(link_masks[-1].shape[0]))[plaq_index])
    active_phase, orientations, orients_to_shift = alternate_params(
        active_phase,
        width,
        dimensions,
        orientations,
        orients_to_shift)
    
plot_masks(link_masks, plaq_masks, plaq_dirs, opacity=0.25)

# end